In [ ]:
import duckdb
import os
import pandas as pd
import urllib.request 
from zipfile import ZipFile 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.cluster import KMeans


from matplotlib import pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
path_to_export = "/Users/paul_gwu/code/data/MovieLensExport/" ## please replace with your own path!
duckdb.sql(f'''IMPORT DATABASE '{path_to_export}' ''') # parquet is a fast and efficient format
#Let's check that the data all loaded!
duckdb.sql('SHOW TABLES')

In [ ]:
duckdb.sql('SELECT COUNT(DISTINCT movieId), COUNT(DISTINCT UserId) FROM ratings_fact WHERE rating >= 5')

In [ ]:
query = '''

CREATE OR REPLACE TABLE driver_table AS 
(
    SELECT 
        movieId
        , COUNT(*) AS number_reviews
        , AVG(rating) AS average_review
    FROM
        ratings_fact r
    GROUP BY
        movieId
    HAVING number_reviews >=25
)

'''

duckdb.sql(query)
duckdb.sql('SELECT * FROM driver_table').df()
    

In [ ]:
query = '''
CREATE OR REPLACE TABLE driver2 AS (
    WITH genre_rating AS (
    
        SELECT 
              movieId
            , title
            , genres
            , g.genre
        FROM
            movies d
        LEFT JOIN
            genre_dim g
        USING (movieId)

        
    )
    , genre_features AS (
        PIVOT genre_rating
        ON  genre 
        USING 
         COUNT(genre) AS genre_feature
    )
    
    SELECT
        *
        FROM 
            driver_table d
        LEFT JOIN
            genre_features gf
        USING (movieId)
)
'''
duckdb.sql(query)
duckdb.sql("SELECT * FROM driver2 LIMIT 10").df()

In [ ]:
df = duckdb.sql("SELECT * FROM driver2 ").df()
features = [i for i in df.columns if 'genre_feature' in i]
X = df[features]

kmeans = KMeans(n_clusters=20, random_state=0, n_init="auto").fit(X)


In [ ]:
df[kmeans.labels_==0]